In [1]:
import sys
import os

debug = False

cwd = os.getcwd()
if cwd not in sys.path:
    sys.path.append(cwd)

import xml.etree.ElementTree as etree
from modulosXMILE.Model import *

source_xmlns = "{http://docs.oasis-open.org/xmile/ns/XMILE/v1.0}" 
source_xmlns_isee = "{http://iseesystems.com/XMILE}"

DIR_XMILE       = 'lotka-volterra-nested/lotka-volterra-nested-2.xmile'
###################################################################################################################
parser = etree.XMLParser(encoding="utf-8")
with open(DIR_XMILE, 'r') as xml_file:
    xml_tree = etree.parse(xml_file, parser=parser)
root = xml_tree.getroot()
models = root.findall(source_xmlns + 'model')
models_parsed = list(map(lambda x : Model(x, debug), models))
for model in models_parsed:
    print model.getName()

top
PredatorModel
PreyModel
PreySubModel
PreySubSubModel


In [2]:
from jinja2 import DictLoader, Environment, FileSystemLoader
import os
PATH = './'
TEMPLATE_ENVIRONMENT = Environment(
    autoescape=False,
    loader=FileSystemLoader(os.path.join(PATH, 'templates')),
    trim_blocks=False)
def render_template(template_filename, context):
    return TEMPLATE_ENVIRONMENT.get_template(template_filename).render(context)
devsml_template_filename = 'template-devsml.xml'

# Clases DEVS

In [3]:
### TODO : Constant, Aux, Integrator, Ftot y Fpm podrian ser subclase de DEVSAtomicComponent
from modulosDEVS.DEVSCoupledComponent import *

In [8]:
topModel = models_parsed[0]
dm = DEVSCoupledComponent(topModel, models_parsed)

def traverse(dm):
    name = dm.getName()
    print name
    ccs = dm.getDEVSCoupledComponents()
    
    # Genero xml
    # PredatorModel, PreyModel, PreySubModel, PreySubSubModel
    if name == 'DEVS_COUPLED_PreyModel':
        dst_filename = name + '.xml'
        context = {
            'coupled_name' : name,
            'coupled_filenames' : list(map(lambda x : x.getName(), ccs)),
            'atomics' : dm.getDEVSAtomicComponents(),
            'internal_connections' : [],
            'external_input_connections' : [],
            'external_output_connections' : []
        }
        tmpl = render_template(devsml_template_filename, context)
        print tmpl
        
        # TODO : guardar el xml
        
    # Recursion
    for cc in ccs:
        traverse(cc)
traverse(dm)

# TODO : fusionar todos los xml generados en la funcion 'traverse'

DEVS_COUPLED_top
DEVS_COUPLED_PredatorModel
DEVS_BASIC_COUPLED_Predator_Prey_hunter
DEVS_COUPLED_PreyModel
<!-- Acoplado : DEVS_COUPLED_PreyModel -->
<coupled name="DEVS_COUPLED_PreyModel" model="Coupled" >

<!-- Inputs -->
<inputs>
</inputs>

<!-- Outputs -->
<outputs>
</outputs>

<!-- Components -->
<components>
<!-- Atomico : paramB -->
<atomicRef name="paramB" type="DEVSConstant">
</atomicRef>
<!-- Atomico : paramA -->
<atomicRef name="paramA" type="DEVSConstant">
</atomicRef>
__INCLUDE__DEVS_COUPLED_PreySubModel
__INCLUDE__DEVS_BASIC_COUPLED_Prey_Predator_PreySubModelStock_PreySubSubModelStock
</components>

<!-- Internal Connections -->
<internal_connections>

</internal_connections>

<!-- External Input Connections -->
<external_input_connections>

</external_input_connections>

<!-- External Output Connections -->
<external_output_connections>

</external_output_connections>

</coupled>
DEVS_COUPLED_PreySubModel
DEVS_COUPLED_PreySubSubModel
DEVS_BASIC_COUPLED_PreySubSubModelSto